In [12]:
#logistic regression

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# Load the data
data = pd.read_csv('pseudodata_præoperation.csv')

#make pd to np
X = data.to_numpy()
y = np.random.choice([0, 1], size=len(data))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

In [14]:
# Define the Logistic Regression Model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))
    
input_dim = X_train.shape[1]

def train_and_evaluate_model(X_train, y_train, X_val, y_val, input_dim, num_epochs=100, lr=0.01):
    model = LogisticRegressionModel(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate on validation set
    with torch.no_grad():
        predicted = model(X_val).round()
        accuracy = (predicted.eq(y_val).sum() / float(y_val.shape[0])).item()
    
    return accuracy

outer_kf = KFold(n_splits=10, shuffle=True, random_state=42)
outer_accuracies = []

for outer_train_index, outer_test_index in outer_kf.split(X):
    X_outer_train, X_outer_test = X[outer_train_index], X[outer_test_index]
    y_outer_train, y_outer_test = y[outer_train_index], y[outer_test_index]

    inner_kf = KFold(n_splits=10, shuffle=True, random_state=42)
    inner_accuracies = []

    for inner_train_index, inner_val_index in inner_kf.split(X_outer_train):
        X_inner_train, X_inner_val = X_outer_train[inner_train_index], X_outer_train[inner_val_index]
        y_inner_train, y_inner_val = y_outer_train[inner_train_index], y_outer_train[inner_val_index]

        accuracy = train_and_evaluate_model(X_inner_train, y_inner_train, X_inner_val, y_inner_val, input_dim)
        inner_accuracies.append(accuracy)
        print(f'Inner Fold Accuracy: {accuracy * 100:.2f}%')

    mean_inner_accuracy = sum(inner_accuracies) / len(inner_accuracies)
    print(f'Outer Fold Mean Inner Accuracy: {mean_inner_accuracy * 100:.2f}%')

    outer_accuracy = train_and_evaluate_model(X_outer_train, y_outer_train, X_outer_test, y_outer_test, input_dim)
    outer_accuracies.append(outer_accuracy)
    print(f'Outer Fold Accuracy: {outer_accuracy * 100:.2f}%')

print(f'Mean Outer Accuracy: {sum(outer_accuracies)/len(outer_accuracies) * 100:.2f}%')



Inner Fold Accuracy: 49.48%
Inner Fold Accuracy: 47.04%
Inner Fold Accuracy: 49.13%
Inner Fold Accuracy: 48.43%
Inner Fold Accuracy: 48.08%
Inner Fold Accuracy: 52.61%
Inner Fold Accuracy: 46.85%
Inner Fold Accuracy: 51.75%
Inner Fold Accuracy: 55.24%
Inner Fold Accuracy: 48.60%
Outer Fold Mean Inner Accuracy: 49.72%
Outer Fold Accuracy: 51.72%
Inner Fold Accuracy: 53.66%
Inner Fold Accuracy: 54.36%
Inner Fold Accuracy: 54.70%
Inner Fold Accuracy: 39.72%
Inner Fold Accuracy: 51.92%
Inner Fold Accuracy: 50.52%
Inner Fold Accuracy: 46.15%
Inner Fold Accuracy: 48.60%
Inner Fold Accuracy: 48.25%
Inner Fold Accuracy: 49.30%
Outer Fold Mean Inner Accuracy: 49.72%
Outer Fold Accuracy: 50.16%
Inner Fold Accuracy: 55.40%
Inner Fold Accuracy: 50.87%
Inner Fold Accuracy: 53.31%
Inner Fold Accuracy: 49.48%
Inner Fold Accuracy: 53.66%
Inner Fold Accuracy: 51.22%
Inner Fold Accuracy: 49.30%
Inner Fold Accuracy: 55.59%
Inner Fold Accuracy: 53.15%
Inner Fold Accuracy: 48.25%
Outer Fold Mean Inner Accu